Подготовка датасета из JSON

In [ ]:
import json
from pathlib import Path
import os

# train labels
json_path = 'InsPLAD-det/annotations/instances_train.json'
annotations_path = 'InsPLAD-det/labels/train'
#val labels
# json_path = 'InsPLAD-det/annotations/instances_val.json'
# annotations_path = 'InsPLAD-det/labels/val'
os.makedirs(annotations_path, exist_ok=True)

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

image_dict = {img['id']: img for img in data['images']}


for ann in data['annotations']:
    img_id = ann['image_id']
    img = image_dict[img_id]
    file_name = Path(img['file_name']).stem  # без .jpg
    txt_path = os.path.join(annotations_path, f"{file_name}.txt")

    ann_class = ann['category_id']
    # bbox in COCO
    x_min, y_min, w, h = ann['bbox']
    # --- Нормализация в YOLO ---
    img_w, img_h = img['width'], img['height']
    x_center = (x_min + w / 2) / img_w
    y_center = (y_min + h / 2) / img_h
    w_norm = w / img_w
    h_norm = h / img_h

    with open(txt_path, "w", encoding='utf-8') as f:
        f.write(f"{ann_class} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")

Обучение YOLOv8m 

In [1]:
from ultralytics import YOLO

modely = YOLO('yolov8m.pt')

results = modely.train(
    data="Datasets/InsPLAD-det/data.yaml",
    epochs=80,
    imgsz=640,
    batch=32,
    device=0,
    amp=True,
    cache=True,
    freeze=5
)

New https://pypi.org/project/ultralytics/8.3.225 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.221 🚀 Python-3.13.5 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7781MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Datasets/InsPLAD-det/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=5, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nm

In [5]:
import os
import cv2
from ultralytics import YOLO
model = YOLO('models/last.pt')
path = 'Datasets/yoke-suspension/train/'
data = 'good'
data_path = os.path.join(path, data)
results = model(data_path)
res_dir = os.path.join(path, 'labels')
os.makedirs(res_dir, exist_ok=True)

for r, image in zip(results, os.listdir(data_path)):
        bbox = r.boxes.xywh
        # name = os.path.join(res_dir, image)
        # with open(name, 'w', encoding='utf-8') as f: 
        #         f.write()

        



image 1/299 /home/cyberpsychoz/RailRoad_Energy_Communications_Tracker/Datasets/yoke-suspension/train/good/Fotos 01-12-2020_DJI_0019_amarra_balancim_336.jpg: 640x640 1 2, 19.7ms
image 2/299 /home/cyberpsychoz/RailRoad_Energy_Communications_Tracker/Datasets/yoke-suspension/train/good/Fotos 01-12-2020_DJI_0134_amarra_balancim_274.jpg: 640x640 (no detections), 15.6ms
image 3/299 /home/cyberpsychoz/RailRoad_Energy_Communications_Tracker/Datasets/yoke-suspension/train/good/Fotos 01-12-2020_DJI_0155_amarra_balancim_289.jpg: 640x640 (no detections), 13.6ms
image 4/299 /home/cyberpsychoz/RailRoad_Energy_Communications_Tracker/Datasets/yoke-suspension/train/good/Fotos 01-12-2020_DJI_0195_amarra_balancim_245.jpg: 640x640 (no detections), 14.3ms
image 5/299 /home/cyberpsychoz/RailRoad_Energy_Communications_Tracker/Datasets/yoke-suspension/train/good/Fotos 01-12-2020_DJI_0293_amarra_balancim_222.jpg: 640x640 1 8, 12.1ms
image 6/299 /home/cyberpsychoz/RailRoad_Energy_Communications_Tracker/Datasets